Stock Market Prediction And Forecasting Using Stacked LSTM

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [17]:
# Load the stock market dataset
dataset_url = 'https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv'
df = pd.read_csv(dataset_url)

In [18]:
# Preprocess the dataset
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', inplace=True, ascending=True)
df.reset_index(drop=True, inplace=True)
dataset = df['Close'].values.reshape(-1, 1)

In [19]:
# Split the dataset into training and testing sets
train_data = dataset[:int(0.8 * len(dataset))]
test_data = dataset[int(0.8 * len(dataset)):]

In [20]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)

In [21]:
# Prepare the training data
X_train = []
y_train = []
for i in range(100, len(train_data_scaled)):
    X_train.append(train_data_scaled[i-100:i, 0])
    y_train.append(train_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [22]:
# Reshape the training data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [23]:
# Build the Stacked LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [24]:
# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
48/48 [==============================] - 9s 99ms/step - loss: 0.0267
Epoch 2/10
48/48 [==============================] - 5s 112ms/step - loss: 0.0054
Epoch 3/10
48/48 [==============================] - 5s 98ms/step - loss: 0.0050
Epoch 4/10
48/48 [==============================] - 5s 104ms/step - loss: 0.0045
Epoch 5/10
48/48 [==============================] - 5s 111ms/step - loss: 0.0045
Epoch 6/10
48/48 [==============================] - 5s 100ms/step - loss: 0.0041
Epoch 7/10
48/48 [==============================] - 5s 112ms/step - loss: 0.0035
Epoch 8/10
48/48 [==============================] - 5s 100ms/step - loss: 0.0035
Epoch 9/10
48/48 [==============================] - 5s 112ms/step - loss: 0.0033
Epoch 10/10
48/48 [==============================] - 5s 99ms/step - loss: 0.0030


In [25]:
# Prepare the test data
inputs = df[len(df) - len(test_data) - 100:]['Close'].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)
X_test = []
for i in range(100, len(inputs)):
    X_test.append(inputs[i-100:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [26]:
# Predict stock prices
predicted_stock_prices = model.predict(X_test)
predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)

13/13 [==============================] - 1s 26ms/step


In [27]:
# Evaluate the model
rmse = np.sqrt(np.mean((predicted_stock_prices - test_data)**2))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 31.19551067952075
